# MNIST Classifier with HuoguoML

This short notebook shows how to use HuoguoML to deploy and manage a service. This includes:

1. Building and training a neural network to classify MNIST images
2. Create a HuoguoML service

## Requirements

We will be using Tensorflow 2 and HuoguoML. Update `pip` and install packages:

In [ ]:
!pip install --upgrade pip
!pip install tensorflow
!pip install huoguoml

Import all packages:

In [ ]:
import tensorflow as tf

## Building and training a neural network to classify MNIST images

### Data

First we load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Further, the samples are converted from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### Create a neural network
A two layer fully connected neural network, that is trained with cross entropy loss and optimized with adam model:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### Train, evaluate and export neural network

Train neural network

In [ ]:
model.fit(x_train, y_train, epochs=5)

Evaluate neural network -the image classifier should have a accuracy of ~98% on this dataset.

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

Export neural network

In [ ]:
model_dir = "./model"
model.save(model_dir)

In [ ]:
!saved_model_cli show --dir ./model --all

In [ ]:
!pip uninstall huoguoml

## Create a HuoguoML service

In [1]:
import sys
sys.path.insert(0,'..')
import huoguoml
import time
import random

In [51]:
run = huoguoml.start_experiment_run("mnist-15", "127.0.0.1:8080")

In [52]:
run.log_parameter("batch_size", random.randint(0, 100))


In [53]:
run.log_metric("batch_size", random.randint(0, 100))
run.log_tag("framework", "tensorflow")
run.log_tag("git", "5123131241ed")

In [54]:
run.end_experiment_run(failed=False)

In [55]:
run.run

Run(experiment_name='mnist-15', author='stevenmi', status=1, id=5, run_nr=5, creation_time=1619386672.87073, finish_time=1619386673.275448, duration=0.40471816062927246, description='', parameters={'batch_size': 63}, metrics={'batch_size': 53}, tags={'framework': 'tensorflow', 'git': '5123131241ed'}, model_definition=None)

In [58]:
with huoguoml.start_experiment_run("mnist-5", "http://127.0.0.1:8080") as run:
    run.log_parameter("batch_size", random.randint(0, 100))
    run.log_parameter("alpha", random.randint(0, 100))
    run.log_parameter("beta", random.randint(0, 100))
    run.log_parameter("gamma", random.randint(0, 100))
    run.log_parameter("delta", random.randint(0, 100))
    run.log_metric("accuracy", random.randint(0, 100))
    run.log_metric("precision", random.randint(0, 100))
    run.log_metric("recall", random.randint(0, 100))
    run.log_metric("f1", random.randint(0, 100))
    run.log_tag("framework", "tensorflow")
    run.log_tag("git", "5123131241ed")

In [27]:
with huoguoml.start_experiment_run("mnist-15", "http://127.0.0.1:8080") as run:
    run.log_parameter("batch_size", random.randint(0, 100))
    run.log_parameter("alpha", random.randint(0, 100))
    run.log_parameter("beta", random.randint(0, 100))
    run.log_parameter("gamma", random.randint(0, 100))
    run.log_parameter("delta", random.randint(0, 100))
    run.log_metric("accuracy", random.randint(0, 100))
    run.log_metric("precision", random.randint(0, 100))
    run.log_metric("recall", random.randint(0, 100))
    run.log_metric("f1", random.randint(0, 100))
    run.log_tag("framework", "tensorflow")
    run.log_tag("git", "5123131241ed")
    run.log_model("tensorflow", tf_saved_model_dir="../model",
                                tf_meta_graph_tags="serve",
                                tf_signature_def_key="serving_default")

ImportError: cannot import name 'ModelNode' from 'huoguoml.schemas' (../huoguoml/schemas/__init__.py)

In [ ]:
for i in range(15):
    with huoguoml.start_experiment_run("mnist-15", artifact_dir="../huoguoml") as run:
        run.log_parameter("batch_size", random.randint(0, 100))
        run.log_parameter("alpha", random.randint(0, 100))
        run.log_parameter("beta", random.randint(0, 100))
        run.log_parameter("gamma", random.randint(0, 100))
        run.log_parameter("delta", random.randint(0, 100))
        run.log_metric("accuracy", random.randint(0, 100))
        run.log_metric("precision", random.randint(0, 100))
        run.log_metric("recall", random.randint(0, 100))
        run.log_metric("f1", random.randint(0, 100))
        run.log_tag("framework", "tensorflow")
        run.log_tag("git", "5123131241ed")
        run.log_model("tensorflow", tf_saved_model_dir="../model",
                                  tf_meta_graph_tags="serve",
                                  tf_signature_def_key="serving_default")
        time.sleep(random.randint(0, 5))

In [ ]:
for i in range(3):
    with huoguoml.start_experiment_run("mnist-3", artifact_dir="../huoguoml") as run:
        run.log_parameter("batch_size", random.randint(0, 100))
        run.log_parameter("alpha", random.randint(0, 100))
        run.log_parameter("beta", random.randint(0, 100))
        run.log_parameter("gamma", random.randint(0, 100))
        run.log_parameter("delta", random.randint(0, 100))
        run.log_metric("accuracy", random.randint(0, 100))
        run.log_metric("precision", random.randint(0, 100))
        run.log_metric("recall", random.randint(0, 100))
        run.log_metric("f1", random.randint(0, 100))
        run.log_tag("framework", "tensorflow")
        run.log_tag("git", "5123131241ed")
        run.log_model("tensorflow", tf_saved_model_dir="../model",
                                  tf_meta_graph_tags="serve",
                                  tf_signature_def_key="serving_default")
        time.sleep(random.randint(0, 5))

In [ ]:
for i in range(8):
    with huoguoml.start_experiment_run("mnist-8", artifact_dir="../huoguoml") as run:
        run.log_parameter("batch_size", random.randint(0, 100))
        run.log_parameter("alpha", random.randint(0, 100))
        run.log_parameter("beta", random.randint(0, 100))
        run.log_parameter("gamma", random.randint(0, 100))
        run.log_parameter("delta", random.randint(0, 100))
        run.log_metric("accuracy", random.randint(0, 100))
        run.log_metric("precision", random.randint(0, 100))
        run.log_metric("recall", random.randint(0, 100))
        run.log_metric("f1", random.randint(0, 100))
        run.log_tag("framework", "tensorflow")
        run.log_tag("git", "5123131241ed")
        run.log_model("tensorflow", tf_saved_model_dir="../model",
                                  tf_meta_graph_tags="serve",
                                  tf_signature_def_key="serving_default")
        time.sleep(random.randint(0, 5))